In [1]:
import matplotlib.pyplot as plt
import pandas as pd
#import modin.pandas as pd
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import gc
from sklearn.metrics import confusion_matrix, roc_curve 
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_colwidth', None)
pd.set_option('future.no_silent_downcasting', True)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# df = pd.read_csv("output_adam.csv")
# df_label = pd.read_csv("mirai_labels.csv")
# df["alert"] = df_label
# df.head()

In [2]:
df = pd.read_csv("df_train.csv")

2025-05-19 16:54:39,239	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.3 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-05-19 16:54:41,069	INFO worker.py:1819 -- Started a local Ray instance.
2025-05-19 16:54:41,691	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.3 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.


In [3]:
df.columns

Index(['frame.number', 'frame.len', 'frame.time', 'frame.time_epoch',
       'frame.protocols', 'eth.src', 'eth.dst', 'eth.type', 'ip.src', 'ip.dst',
       'ip.len', 'ip.ttl', 'ip.flags', 'ip.frag_offset', 'ip.proto',
       'ip.version', 'ip.dsfield', 'ip.checksum', 'tcp.srcport', 'tcp.dstport',
       'tcp.len', 'tcp.seq', 'tcp.ack', 'tcp.flags', 'tcp.flags.syn',
       'tcp.flags.ack', 'tcp.flags.fin', 'tcp.flags.reset', 'tcp.window_size',
       'tcp.checksum', 'tcp.stream', 'udp.srcport', 'udp.dstport',
       'udp.length', 'udp.checksum', 'icmp.type', 'icmp.code', 'icmp.checksum',
       'http.request.method', 'http.request.uri', 'http.request.version',
       'http.request.full_uri', 'http.response.code', 'http.user_agent',
       'http.content_length_header', 'http.content_type', 'http.cookie',
       'http.host', 'http.referer', 'http.location', 'http.authorization',
       'http.connection', 'dns.qry.name', 'dns.qry.type', 'dns.qry.class',
       'dns.flags.response', 'dns.f

In [3]:
df["frame.time"].iloc[1]

'Aug  9, 2017 10:57:35.416698000 Środkowoeuropejski czas letni'

In [4]:
def ordinalize_packets(packet_data, column="ip"):
    src_col = f"{column}.src"
    dst_col = f"{column}.dst"
    if 'frame.number' not in packet_data.columns or src_col not in packet_data.columns or dst_col not in packet_data.columns:
        raise ValueError("DataFrame must contain 'frame.number' and  columns")

    out_src = packet_data.reset_index().groupby([src_col])['index'].min()
    out_dst = packet_data.reset_index().groupby([dst_col])['index'].min() + 0.5
    out = pd.concat([out_src, out_dst], axis=0).sort_values()
    out = out[~out.index.duplicated(keep='first')]
    out.iloc[:] = out.reset_index(name="ip").index
    out = out.astype(int)
    mappings = out.to_dict()
    return mappings
    #packet_data.loc[:, src_col] = packet_data[src_col].replace(mappings).astype(int)
    #packet_data.loc[:, dst_col] = packet_data[dst_col].replace(mappings).astype(int)

In [5]:
features = ["frame.number", "frame.len", 'frame.time', 'eth.src', 'eth.dst', 'ip.src', 'ip.dst', 'ip.len', 'ip.ttl',
            'tcp.srcport', 'tcp.dstport',
            'tcp.flags.ack', 'tcp.flags.fin', 'tcp.flags.reset', "alert", "udp.srcport", "udp.dstport"]
df = df[features]
df["frame.len"] /= 65535
df["ip.len"] /= 65535
df["ip.ttl"] /= 255
df["tcp.srcport"] /= 65535
df["tcp.dstport"] /= 65535
df["udp.srcport"] /= 65535
df["udp.dstport"] /= 65535
df["frame.time"] = df["frame.time"].str.replace(" Środkowoeuropejski czas letni", "")#mój PCAP był po POlsku Pudzian byłby dumny :peepoGlad# CEST
df["frame.time"] = df["frame.time"].str.replace(" Środkowoeuropejski czas stand.", "")
df["frame.time"] = df["frame.time"].str.replace(" CEST", "")
df["frame.time"] = pd.to_datetime(df["frame.time"])
df['dow_sin'] = np.sin(2 * np.pi * df["frame.time"].dt.dayofweek / 6)
df['dow_cos'] = np.cos(2 * np.pi * df["frame.time"].dt.dayofweek / 6)
df['hour_sin'] = np.sin(2 * np.pi * df["frame.time"].dt.hour / 24.0)
df['hour_cos'] = np.cos(2 * np.pi * df["frame.time"].dt.hour / 24.0)
df["minute_sin"] = np.sin(2 * np.pi * df["frame.time"].dt.minute / 60)
df["minute_cos"] = np.cos(2 * np.pi * df["frame.time"].dt.minute / 60)
df["second_sin"] = np.sin(2 * np.pi * df["frame.time"].dt.second / 60)
df["second_cos"] = np.cos(2 * np.pi * df["frame.time"].dt.second / 60)
df["ms_sin"] = np.sin(2 * np.pi * df["frame.time"].dt.microsecond / 1000000)
df["ms_cos"] = np.cos(2 * np.pi * df["frame.time"].dt.microsecond / 1000000)
df = df.fillna(0)  # only tcp is left
df = df.drop(columns=["frame.time"])

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.
(_remote_exec_multi_chain pid=15529) time data "Feb 25, 2017 10:06:21.412248000" doesn't match format "%B  %d, %Y %H:%M:%S.%f", at position 69996. You might want to try:
(_remote_exec_multi_chain pid=15529)     - passing `format` if your strings have a consistent format;
(_remote_exec_multi_chain pid=15529)     - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
(_remote_exec_multi_chain pid=15529)     - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.. fn=<function Map.register.<locals>.caller.<locals>.<lambda> at 0x7f99c74eed40>, obj=                               frame.time
(_remote_exec_multi_chain pid=15529) 12549135  May  4, 2017 22:17:00.461601000
(_remote_exec_multi_chain pid=15529) 12549136  May  4, 2017 22:09:53.993326000

RayTaskError(ValueError): [36mray::_remote_exec_single_chain()[39m (pid=15529, ip=192.168.0.13)
  At least one of the input arguments for this task could not be computed:
ray.exceptions.RayTaskError: [36mray::_remote_exec_multi_chain()[39m (pid=15529, ip=192.168.0.13)
  At least one of the input arguments for this task could not be computed:
ray.exceptions.RayTaskError: [36mray::_remote_exec_multi_chain()[39m (pid=15529, ip=192.168.0.13)
  File "/usr/local/lib/python3.11/dist-packages/modin/core/execution/ray/common/deferred_execution.py", line 594, in exec_func
    return fn(obj, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/modin/core/dataframe/algebra/map.py", line 65, in <lambda>
    lambda x: function(x, *args, **kwargs), *call_args, **call_kwds
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/modin/core/storage_formats/pandas/query_compiler.py", line 1084, in <lambda>
    lambda

In [8]:
df.head()

,frame.number,frame.len,eth.src,eth.dst,ip.src,ip.dst,ip.len,ip.ttl,tcp.srcport,tcp.dstport,...,dow_sin,dow_cos,hour_sin,hour_cos,minute_sin,minute_cos,second_sin,second_cos,ms_sin,ms_cos
0,1236045,0.023102,ac:9b:0a:fc:2a:73,00:a0:de:f1:88:6e,192.168.2.15,192.168.100.5,0.022889,0.25098,0.0,0.0,...,0.866025,-0.5,0.258819,-0.965926,0.669131,7.431448e-01,1.000000e+00,2.832769e-16,-0.377139,-0.926157
1,84707,0.003189,10:4f:a8:70:22:7e,00:a0:de:f1:88:6e,192.168.2.13,192.168.100.5,0.002976,0.25098,0.0,0.0,...,0.866025,-0.5,0.500000,-0.866025,-0.309017,9.510565e-01,-5.000000e-01,-8.660254e-01,0.499829,-0.866124
2,679170,0.005554,10:4f:a8:70:22:7e,00:a0:de:f1:88:6e,192.168.2.13,192.168.100.5,0.005341,0.25098,0.0,0.0,...,0.866025,-0.5,0.258819,-0.965926,0.207912,9.781476e-01,7.431448e-01,-6.691306e-01,-0.122496,0.992469
3,2245698,0.023102,10:4f:a8:70:22:7e,00:a0:de:f1:88:6e,192.168.2.13,192.168.100.5,0.022889,0.25098,0.0,0.0,...,0.866025,-0.5,0.258819,-0.965926,0.951057,-3.090170e-01,5.665539e-16,-1.000000e+00,0.266260,0.963901
4,2017908,0.000916,10:4f:a8:70:22:7e,00:a0:de:f1:88:6e,192.168.2.13,192.168.100.5,0.000687,0.25098,0.0,0.0,...,0.866025,-0.5,0.258819,-0.965926,1.000000,2.832769e-16,-9.945219e-01,1.045285e-01,0.817640,0.575730


In [38]:
from torch.utils.data import Dataset
from tqdm import trange
from multiprocessing import Pool
import pickle

class NetworkDataset(Dataset):
    def __init__(self, data, window=128, mapping_ip=None, mapping_eth=None):
        p = Pool(processes=8, maxtasksperchild=10)
        self.data = data
        self.window = window
        if mapping_ip is None:
            self.mappings_ip = p.map(self.ordinal_ip, trange(len(self)))
        else:
            self.mappings_ip = pickle.load(open(mapping_ip, "rb"))
        if mapping_eth is None:
            self.mappings_eth = p.map(self.ordinal_eth, trange(len(self)))
        else:
            self.mappings_eth = pickle.load(open(mapping_eth, "rb"))
        p.close()
        p.join()
            
    def ordinal_ip(self, i):
        return ordinalize_packets(self.data[i:i+self.window], "ip")
    def ordinal_eth(self, i):
        return ordinalize_packets(self.data[i:i+self.window], "eth")
    
    def __getitem__(self, index):
        packets = self.data[index:index + self.window]
        labels = packets['alert'].to_numpy().astype(np.float32)
        packets.loc[:, "eth.src"] = packets["eth.src"].map(self.mappings_eth[index]).fillna(0).astype(int)
        packets.loc[:, "eth.dst"] = packets["eth.dst"].map(self.mappings_eth[index]).fillna(0).astype(int)
        packets.loc[:, "ip.src"] = packets["ip.src"].map(self.mappings_ip[index]).fillna(0).astype(int)
        packets.loc[:, "ip.dst"] = packets["ip.dst"].map(self.mappings_ip[index]).fillna(0).astype(int)
        packets = packets.drop(columns=['alert', "frame.number"]).fillna(0)
        return (packets[["eth.src", "eth.dst"]].astype(int).values, packets[["ip.src", "ip.dst"]].astype(int).values,
                packets.drop(columns=["ip.src", "ip.dst", "eth.src", "eth.dst"]).values.astype("float32"), labels)

    def __len__(self):
        return self.data.shape[0] - self.window

TypeError: The decorated object is not a function (got type <class 'type'>).

In [21]:
from torch.nn.utils.parametrizations import weight_norm


class ResBlock(nn.Module):
    def __init__(self, channels, kernel_size, dilation):
        super(ResBlock, self).__init__()
        self.conv1 = weight_norm(
            nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=kernel_size, dilation=dilation))
        self.conv2 = weight_norm(
            nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=kernel_size, dilation=dilation))
        self.pad = nn.ZeroPad1d((kernel_size - 1) // 2 * dilation)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        t = self.pad(x)
        t = F.relu(self.conv1(t))
        t = self.pad(t)
        t = F.relu(self.conv2(t))
        return self.dropout(t) + x

In [22]:
class TCN(nn.Module):
    def __init__(self, channels, kernel_size, blocks):
        super(TCN, self).__init__()
        self.blocks = nn.Sequential(*[ResBlock(channels, kernel_size, 2**i) for i in range(blocks)])
        
    def forward(self, x):
        return self.blocks(x)

In [16]:
rb =TCN(5, 5, 2)
rb(torch.randn(2, 5, 50))

tensor([[[ 0.4930,  2.8734,  0.7040,  1.4007,  0.5570,  0.8242,  0.5057,
           0.6803,  0.7938,  0.3710, -0.5096,  0.6751, -0.0863, -0.2357,
           0.1171,  1.3098, -0.0148,  1.3838, -0.1150, -1.2456, -0.6741,
           0.1877, -0.4860,  1.2179,  2.0754, -0.3600, -0.2070, -0.7151,
           0.6399, -0.0777,  0.3021, -0.3402,  0.4434,  1.5218,  0.5087,
           0.1899, -0.6611, -0.4585,  2.4599,  0.4934,  0.4422,  0.5624,
           2.2803, -1.5272,  0.7496, -0.0379,  1.2823,  0.8802, -0.7981,
           0.6152],
         [ 1.8333, -0.5736, -1.3496,  1.0371,  1.0337,  0.0684, -0.2696,
           2.1811,  0.2497,  1.6957, -2.0771,  0.3720,  2.0790, -0.3636,
           2.0944, -0.1222,  2.0948,  0.6292,  1.3397,  0.6612, -0.6909,
           1.0906, -1.5022,  0.5549, -0.8636, -1.5973, -0.5547,  1.2952,
           1.8265,  0.9957,  2.0427, -0.8961,  1.0709, -0.0629,  0.7529,
          -2.8566,  0.5028, -0.9615, -0.4447, -1.2141,  0.3986,  0.6234,
          -1.0600, -0.6818,  0.

In [30]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.eth_embed = nn.Embedding(128, 32)
        self.ip_embed = nn.Embedding(128, 32)
        self.total_embed = nn.Linear(148, 64)
        self.conv = TCN(128, 5, 3)
        self.projection = nn.Linear(64, 1)

    def forward(self, eth, ip, packet):
        B = packet.size(0)
        L = packet.size(1)
        eth = self.eth_embed(eth).view(B, L, -1)
        ip = self.ip_embed(ip).view(B, L, -1)
        pk_embed = self.total_embed(torch.cat((eth, ip, packet), dim=-1))
        processed = nn.functional.relu(self.conv(pk_embed))
        preds = self.projection(processed).squeeze(-1)
        return nn.functional.sigmoid(preds)

In [16]:
gc.collect()

1004

In [36]:
dataset = NetworkDataset(df, 128, 'ip_mappings_train', 'eth_mappings_train') # preparation of dataset

In [15]:
import pickle
pickle.dump(dataset.mappings_ip, open('ip_mappings_train', 'wb'))
pickle.dump(dataset.mappings_eth, open('eth_mappings_train', 'wb'))

In [37]:
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm
from accelerate import Accelerator

accelerator = Accelerator(mixed_precision="bf16")
model = Model()
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True, num_workers=32, pin_memory=True,
                        persistent_workers=True)
optimizer = Adam(model.parameters(), lr=1e-4)
w = torch.tensor([5.28], dtype=torch.float32, device="cuda")
loss_fn = nn.BCELoss(weight=w)

model, dataloader, optimizer = accelerator.prepare(model, dataloader, optimizer)

for epoch in range(20):
    count = 0
    total = 0

    for eth, ip, pkt, labels in tqdm(dataloader):
        optimizer.zero_grad()
        pkt = model(eth, ip, pkt)
        loss = loss_fn(pkt, labels)
        count += 1
        total += loss.item()
        loss.backward()
        optimizer.step()

    print("Epoch {} Loss {}".format(epoch, total / count))

  0%|          | 0/262717 [00:14<?, ?it/s]


TypingError: Caught TypingError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/usr/local/lib/python3.11/dist-packages/numba/core/dispatcher.py", line 423, in _compile_for_args
    error_rewrite(e, 'typing')
  File "/usr/local/lib/python3.11/dist-packages/numba/core/dispatcher.py", line 364, in error_rewrite
    raise e

In [2]:
df_val = pd.read_csv("df_val.csv")

/tmp/ipykernel_3754/3364097189.py:1: DtypeWarning: Columns (23,24,25,26,27,29,37,38,39,40,41,43,45,47,49,50,51,52,54,55,56,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df_val = pd.read_csv("df_val.csv")


In [7]:
dataset_val = NetworkDataset(df_val, 128)

100%|██████████| 4203397/4203397 [06:22<00:00, 10975.21it/s]


In [9]:
import pickle
pickle.dump(dataset_val.mappings_ip, open('ip_mappings_val', 'wb'))
pickle.dump(dataset_val.mappings_eth, open('eth_mappings_val', 'wb'))

In [14]:
ip, eth, pkt, labels = dataset_val[0]
ip = torch.from_numpy(ip).cuda().unsqueeze(0)
eth = torch.from_numpy(eth).cuda().unsqueeze(0)
pkt = torch.from_numpy(pkt).cuda().unsqueeze(0)
labels = torch.from_numpy(labels).cuda().unsqueeze(0)

pred = model(ip, eth, pkt)
print(pred)
print(pred > 0.5)
labels

NameError: name 'dataset_val' is not defined

In [ ]:
pred = pred.reshape(-1, 1)
labels =  labels.reshape(-1, 1)

In [68]:
loss_fn(pred, labels)

tensor(0.7049, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)

In [ ]:
#confusion matrix
cm = confusion_matrix(labels, pred)
num = labels.shape[0]
norm = cm/num
# Plot with Seaborn
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='.4f', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Incident Grade Prediction')
plt.show()

In [ ]:
#ROC Curve
fpr, tpr, thresholds = roc_curve(labels, pred)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.4f)' % roc_auc_score(y_test, y_pred))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()